In [3]:
import pandas as pd
import numpy as np
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
data = pd.read_csv('train.csv')

In [5]:
import re

In [6]:
data.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [7]:
data.isnull().sum()

id       0
label    0
tweet    0
dtype: int64

In [13]:
ps = PorterStemmer()

In [14]:

def all_the_cooking(line):
    reviews = re.sub('[^a-zA-Z]' , ' ', line)
    reviews = reviews.lower()
    reviews = reviews.split()
    reviews = [ps.stem(word) for word in reviews if not word in set(stopwords.words('english'))]
    reviews = ' '.join(reviews)
    return(reviews)

In [15]:
for i,line in enumerate(data.tweet):
    data.tweet[i] = all_the_cooking(line)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [16]:
data.head()

,id,label,tweet
0,1,0,user father dysfunct selfish drag kid dysfunct...
1,2,0,user user thank lyft credit use caus offer whe...
2,3,0,bihday majesti
3,4,0,model love u take u time ur
4,5,0,factsguid societi motiv


In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data['tweet'], data['label'], test_size=0.5, stratify=data['label'])

trainp=data[data.label==1]
trainn=data[data.label==0]
print(trainp.info())
trainn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2242 entries, 13 to 31960
Data columns (total 3 columns):
id       2242 non-null int64
label    2242 non-null int64
tweet    2242 non-null object
dtypes: int64(2), object(1)
memory usage: 70.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 29720 entries, 0 to 31961
Data columns (total 3 columns):
id       29720 non-null int64
label    29720 non-null int64
tweet    29720 non-null object
dtypes: int64(2), object(1)
memory usage: 928.8+ KB


In [19]:
train_imbalance = data.copy()
df_minority = data[data['label'] == 1]
df_majority = data[data['label'] == 0]


In [21]:
from sklearn.utils import resample
df_minority_resampled = resample(df_minority, replace = True , n_samples = len(df_majority) , random_state =123)

In [23]:
df_upsampled = pd.concat([df_majority, df_minority_resampled])
 
# Display new class counts
print("Before")
print(data.label.value_counts())
print("After")
print(df_upsampled.label.value_counts())

Before
0    29720
1     2242
Name: label, dtype: int64
After
1    29720
0    29720
Name: label, dtype: int64


In [35]:
X_train, X_test, y_train, y_test = train_test_split(df_upsampled['tweet'], df_upsampled['label'], test_size = 0.5 , stratify = df_upsampled['label'])

In [36]:
vect = TfidfVectorizer()

In [37]:
tf_train = vect.fit_transform(X_train)
tf_test = vect.transform(X_test)

In [38]:
from sklearn.naive_bayes import MultinomialNB

In [39]:
model = MultinomialNB()

In [40]:
model.fit(X=tf_train, y = y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [41]:
expected = y_test
predicted=model.predict(tf_test)

In [42]:
tf_test = vect.transform(X_test)

In [43]:
predicted = model.predict(tf_test)

In [34]:
print(tf_train.shape)
print(tf_test.shape)

(29720, 20874)
(29720, 21033)


In [44]:
from sklearn import metrics

print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))


              precision    recall  f1-score   support

           0       0.97      0.90      0.94     14860
           1       0.91      0.98      0.94     14860

    accuracy                           0.94     29720
   macro avg       0.94      0.94      0.94     29720
weighted avg       0.94      0.94      0.94     29720

[[13437  1423]
 [  353 14507]]


In [46]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, predicted)

In [47]:
print(accuracy)

0.940242261103634
